esm based dynamic model (not using static embeds).

+ Use HF Trainer, LORA:
  * https://huggingface.co/blog/AmelieSchreiber/esmbind

Use TF:
*  🇰https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/protein_language_modeling-tf.ipynb#scrollTo=de8419b5
* Torch based /Trainer example:  https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/protein_language_modeling.ipynb#scrollTo=49dcba23



* NOte for pytorch training could use trainer maybe, and mixed precision - https://huggingface.co/docs/transformers/v4.18.0/en/performance#fp16-training


* QLORA finetuning: https://huggingface.co/blog/AmelieSchreiber/esm2-ptm
  * https://huggingface.co/blog/AmelieSchreiber/esmbind   (token level)

* Another lora, qlora example - may use too much mem/bug : https://github.com/huggingface/peft/issues/1023
* Default trainer (`AutoModelForSequenceClassification`) + Lora https://huggingface.co/docs/peft/task_guides/image_classification_lora
   * seq cls with lora - maybe `task_type="SEQ_CLS"` ? https://github.com/huggingface/peft/blob/main/docs/source/task_guides/ptuning-seq-classification.md
* https://www.kaggle.com/code/andregrbnr/protein-sequence-classification - lora modules to save ??

  * ESM2-Lora mem bug (also accel data loop) ? https://github.com/huggingface/peft/issues/1023


* QLORA: https://huggingface.co/blog/AmelieSchreiber/esm2-ptm
  * `36 batch size` with esm-150M !

* lora peft - classifier layer weight saving issue?  https://github.com/huggingface/peft/issues/577

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !conda install -c conda-forge google-colab -y

# !pip install tensorflow  -U -q # ankh

# !pip install torch  -U -q # fair-esm # seqeval
# !pip install transformers peft accelerate datasets evaluate bitsandbytes -U -q # --user
# !pip install peft bitsandbytes -U

* Use the unirpot fasta file I downloaded and uploaded to my drive

`/content/drive/MyDrive/Research/biodata/proteins/Transmembrane_human_90.fasta`

* Download fasta from: `https://www.uniprot.org/uniref/?query=uniprot:(keyword%3A%22Transmembrane+%5BKW-0812%5D%22+AND+organism%3A%22Homo+sapiens+%28Human%29+%5B9606%5D%22)+identity:0.9`

In [3]:
#### DATA_PATH = "/content/drive/MyDrive/Research/CIDR-Protein Anomalies project/protein_anomalies_data/swp_human_viri_all_embed_esm.parquet" ## ESM1B embedding (max len 1022)
# DATA_PATH = "/content/drive/MyDrive/proteins/New Protein-Virus anom project/hum_vir_swp-globalEmbed-train.csv.gz"## TRAIN
# DATA_PATH = "/content/drive/MyDrive/hum_vir_swp-globalEmbed-train.csv.gz"
# DATA_PATH = "/kaggle/input/humvir-proteins/hum_vir_swp-globalEmbed-train.csv/hum_vir_swp-globalEmbed-train.csv"
DATA_PATH = "hum_vir_swp-globalEmbed-train.csv.gz"

## TEST data:
# TEST_DATA_PATH = "/content/drive/MyDrive/proteins/New Protein-Virus anom project/hum_vir_swp-globalEmbed-test.csv.gz"## TRAIN
# TEST_DATA_PATH = "/content/drive/MyDrive/hum_vir_swp-globalEmbed-test.csv.gz"
TEST_DATA_PATH = "hum_vir_swp-globalEmbed-test.csv.gz"

# ## metadata for all reviewed/swissprot human + virus proteins
# METADATA_PATH = "/content/drive/MyDrive/Research/CIDR-Protein Anomalies project/protein_anomalies_data/SWP_human_viruses_all.xlsx"

TARGET_COL = "virus" ## use for filtering data into 1 class

MAX_LEN = 1024#768#530#480 # exclude sequences longer than this. (Not merely truncate)

FAST_RUN = False#True
SAVE_MODEL = False#True
SAVE_OUTPUTS = False#True

# MODEL_DRIVE_SAVE_PATH = "/content/drive/MyDrive/proteins/New Protein-Virus anom project/trained_esm_lora_trainer_model"
MODEL_DRIVE_SAVE_PATH = "small_trained_esm_lora_trainer_model"
# MODEL_DRIVE_SAVE_PATH = "/kaggle/input/humvir-proteins/qlora/qlora" # saved + reuploadedon kaggle

TRAIN_MODEL = True#False
LOAD_TRAINED = False#True

# Embed sequences in a FASTA file

In [4]:
# from bio_embeddings.embed import ProtTransBertBFDEmbedder
# from Bio import SeqIO
import torch
from sklearn.model_selection import StratifiedKFold, GroupKFold, StratifiedGroupKFold
import pandas as pd
import numpy as np
import random
# from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
# from sklearn.svm import OneClassSVM
from sklearn.preprocessing  import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_predict
# from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
# from sklearn.ensemble import HistGradientBoostingClassifier
from tqdm.autonotebook import tqdm
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, roc_auc_score
from transformers import TrainingArguments, Trainer, logging
from accelerate import Accelerator
from torch.utils.data.dataloader import DataLoader
## https://huggingface.co/docs/transformers/perf_train_gpu_one
from torch import nn
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AdamWeightDecay
# from tensorflow.keras.optimizers import Adafactor, Adam # more memory effecient than adamWD
# import tensorflow
# from tensorflow.keras.metrics.AUC()
# from transformers import AutoTokenizer #DataCollatorForLanguageModeling,
## https://huggingface.co/docs/transformers/model_doc/esm#transformers.EsmForSequenceClassification.forward.example
from transformers import AutoTokenizer, EsmForSequenceClassification, Trainer
from transformers import TFAutoModelForSequenceClassification ,TFEsmForSequenceClassification
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

from transformers import (
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
)
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from transformers import AutoTokenizer, BitsAndBytesConfig
from peft import AutoPeftModelForSequenceClassification, AutoPeftModel

## could use transformer pipeline for inference;
import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
# from tqdm.auto import tqdm
# pipe = pipeline("text-classification", model="facebook/wav2vec2-base-960h", device=0)
from datetime import date
today = date.today()

import os
os.environ["WANDB_PROJECT"]="humVir_evalDLTest"

/tmp/ipykernel_93/2867410804.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2024-03-12 09:45:21.262796: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-12 09:45:21.439838: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 09:45:21.439866: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 09:45:21.469938: E external/local_xla/xla/stream_execut

In [5]:
import wandb
# wandb.init(mode="disabled")
# # alt
# wandb.init(project='qlora_humvir')# ; or args = TrainingArguments(report_to="none")

In [6]:
from accelerate import Accelerator
# # Use the accelerator
# ### try disabling? (with qlora)
# # accelerator = Accelerator()# trying this
# # שבבקךקרשאםר צשטנק בשודקד ןדדוקד?
accelerator = Accelerator(mixed_precision="fp16") #"bf16") #bf16") # fp16 # orig used

In [7]:
num_epochs = 3#4
bch_size = 16#36#32#8#3#2

# opt = Adafactor(3e-4)##AdamWeightDecay(1e-4) #default: AdamWeightDecay(2e-5)
# opt = AdamWeightDecay(5e-4)#(1e-3)
# opt = Adam(8e-4)

if FAST_RUN:
    num_epochs = 2
    # bch_size = 16
    bch_size = 32
    MAX_LEN = int(MAX_LEN//1.4)

In [8]:
# model_checkpoint ="facebook/esm2_t6_8M_UR50D"
# model_checkpoint =  "facebook/esm2_t12_35M_UR50D"
# model_checkpoint =  "facebook/esm2_t30_150M_UR50D"
# model_checkpoint =  "facebook/esm2_t33_650M_UR50D"
models_list = ["facebook/esm2_t6_8M_UR50D","facebook/esm2_t12_35M_UR50D","facebook/esm2_t30_150M_UR50D","facebook/esm2_t33_650M_UR50D"]
if FAST_RUN:
#     model_checkpoint ="facebook/esm2_t6_8M_UR50D"
  # model_checkpoint =  "facebook/esm2_t12_35M_UR50D"
    models_list = ["facebook/esm2_t6_8M_UR50D","facebook/esm2_t12_35M_UR50D"]


In [9]:
# df = pd.read_parquet(DATA_PATH) # numpy to pandas
df = pd.read_csv(DATA_PATH,usecols=["Sequence","virus","Length","Cluster name"])
df_test = pd.read_csv(TEST_DATA_PATH,usecols=["Sequence","virus","Length","Cluster name"])
 ## lengths of all the seqs

## concat all data - for CV
# df = pd.concat([df,df_test])
print(df.shape[0])
print(df.nunique())

if FAST_RUN:
#   # df.loc[df["Length"]>200]
    df = df.sample(frac=0.01)
    df_test = df_test.sample(frac=0.02)
df

20340
Sequence        20340
virus               2
Length           1454
Cluster name    16810
dtype: int64


,Sequence,virus,Length,Cluster name
0,MADFLKGLPVYNKSNFSRFHADSVCKASNRRPSVYLPTREYPSEQI...,0,102,Cluster: DET1- and DDB1-associated protein 1
1,MPSTLQVLAKKVLALEHKENDHISREYYYHILKCCGLWWHEAPIIL...,1,362,Cluster: Protein MGF 360-19R
2,MASSAELDFNLQALLEQLSQDELSKFKSLIRTISLGKELQTVPQTE...,0,97,Cluster: Pyrin domain-containing protein 2
3,MAAWGKKHAGKDPVRDECEERNRFTETREEDVTDEHGEREPFAETD...,0,186,Cluster: Protein FAM9B
4,MASDSPARSLDEIDLSALRDPAGIFELVELVGNGTYGQVYKGRHVK...,0,1360,Cluster: TRAF2 and NCK-interacting protein kinase
...,...,...,...,...
20335,MDPDKQDALNSIENSIYRTAFKLQSVQTLCQLDLIDSSLIQQVLLR...,0,578,Cluster: Dystrotelin
20336,MLCPWRTANLGLLLILTIFLVAEAEGAAQPNNSLMLQTSKENHALA...,0,348,Cluster: Cell surface glycoprotein CD200 recep...
20337,MCLRFFSPVPGSTSSATNVTMVVSAGPWSSEKAEMNILEINEKLRP...,0,421,Cluster: Putative neuroblastoma breakpoint fam...
20338,MASHAGQQHAPAFGQAARASGPTDGRAASRPSHRQGASEARGDPEL...,1,376,Cluster: Thymidine kinase


In [10]:
# df = df.loc[df["Length"]>500].sample(500)
# print(df.shape[0])

In [11]:
try:
    df.groupby(["virus"])["Cluster name"].nunique().describe().round(2)
except:()

In [12]:
df.groupby(["virus"])["Length"].describe().round(2)

,count,mean,std,min,25%,50%,75%,max
virus,,,,,,,,
0,14668.0,473.16,308.52,11.0,244.0,398.0,623.0,1533.0
1,5672.0,380.31,298.35,11.0,151.0,297.0,514.0,1520.0


In [13]:
print(df["virus"].agg(["mean","sum","count"]).round(2))
df["Length"].describe().round(0)

mean         0.28
sum       5672.00
count    20340.00
Name: virus, dtype: float64


count    20340.0
mean       447.0
std        309.0
min         11.0
25%        213.0
50%        372.0
75%        592.0
max       1533.0
Name: Length, dtype: float64

In [14]:
# df = df.loc[df["Length"]<=2*MAX_LEN].reset_index(drop=True)
# df_test = df_test.loc[df_test["Length"]<=2*MAX_LEN].reset_index(drop=True)

print(df["Length"].describe().round(1))
print(df["virus"].agg(["mean","sum","count"]).round(2))

count    20340.0
mean       447.3
std        308.5
min         11.0
25%        213.0
50%        372.0
75%        592.2
max       1533.0
Name: Length, dtype: float64
mean         0.28
sum       5672.00
count    20340.00
Name: virus, dtype: float64


In [15]:
df.nunique()

Sequence        20340
virus               2
Length           1454
Cluster name    16810
dtype: int64

In [16]:
# ## metadata about all sequences, can be used to identify and to define targets/labels
# df_meta = pd.read_excel(METADATA_PATH).dropna(how="all",axis=1)
# df_meta

### example pretrained fb/torch:
* https://github.com/facebookresearch/esm#getting-started-with-this-repo-

* Transformers + trainer example : (mlm case): https://github.com/facebookresearch/esm/discussions/556
* keras models supported / via HF?
  * https://huggingface.co/docs/transformers/model_doc/esm#transformers.EsmForSequenceClassification.forward.example


  TF finetuning example (sequence evel?):
  https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/protein_language_modeling-tf.ipynb#scrollTo=4b26b828

In [17]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AdamWeightDecay
from tensorflow.keras.optimizers import Adafactor, Adam # more memory effecient than adamWD
import tensorflow
# from tensorflow.keras.metrics.AUC()
# from transformers import AutoTokenizer #DataCollatorForLanguageModeling,
## https://huggingface.co/docs/transformers/model_doc/esm#transformers.EsmForSequenceClassification.forward.example
from transformers import AutoTokenizer, EsmForSequenceClassification, Trainer
from transformers import TFAutoModelForTokenClassification, TFAutoModelForSequenceClassification ,TFEsmForSequenceClassification

from peft import prepare_model_for_kbit_training, get_peft_model

In [18]:
ID2LABEL = {
    0: "Human",
    1: "Virus"
}
LABEL2ID = {v: k for k, v in ID2LABEL.items()}

In [19]:
train_sequences = df["Sequence"].tolist()
train_labels = df["virus"].tolist()
train_groups = df["Cluster name"].tolist()

# # train_sequences, test_sequences, train_labels, test_labels = train_test_split(sequences, labels, test_size=0.25, shuffle=True)#,stratify=labels)

test_sequences = df_test["Sequence"].tolist()
test_labels = df_test["virus"].tolist()

In [20]:
# peft_config = LoraConfig(base_model_name_or_path=model_checkpoint,
# #                         init_lora_weights="loftq", loftq_config=loftq_config,
#     task_type= "SEQ_CLS",#TaskType.SEQ_CLS, ## disabling helps?? (then get "ValueError: Attempting to unscale FP16 gradients.")
#     inference_mode=False, r= 2 if FAST_RUN else 4, #16,
#     lora_alpha=8,
# #     lora_dropout=0.05,
#     use_rslora = True,
#     bias= "all"#"lora_only",#"none",#"lora_only",#"none",#"all",
#     # target_modules=[
#     #     "query", "key", "value",
#     #                 "EsmSelfOutput.dense",
#     #         "EsmIntermediate.dense",
#     #         "EsmOutput.dense",
#     #                 # "word_embeddings",
#     #                 # "EsmClassificationHead.dense", ## not sure if works/changes anything
#     #                 # "out_proj",
#     #                 # "classifier" # fails
#     #                 ],
# #         target_modules=  target_modules#"all-linear"#modules_list,
#            ,target_modules=  "all-linear"

#     # # ### https://www.kaggle.com/code/andregrbnr/protein-sequence-classification
# #      ,modules_to_save= "all-linear",
# #                          [#"decode_head",
# # #                       "classifier",
# #          "EsmClassificationHead",
# #          'classifier.dense', 'classifier.out_proj',
# #          "pooler",
# # # #                      'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight'
# #                      ]
#     ## 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight'
#     # modules_to_save=["classifier"]
# )

In [21]:
##https://huggingface.co/docs/peft/main/en/developer_guides/quantization
## lotfQ config - for this, do not initialize as quantized!
# from peft import LoftQConfig, LoraConfig, get_peft_model
# loftq_config = LoftQConfig(loftq_bits=4)

## https://huggingface.co/blog/4bit-transformers-bitsandbytes
compute_dtype = getattr(torch, "float16")

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True, # disable to train ok;
#   load_in_8bit=True, # alt
# load_in_4bit=False,
  bnb_4bit_quant_type="nf4",
#    bnb_4bit_use_double_quant=True,
  # llm_int8_has_fp16_weight= True, # try alt
    bnb_4bit_compute_dtype=torch.bfloat16,
)


In [22]:

# Load ESM-2 model
## smallest: esm2_t6_8M_UR50D
## 2d smallest
## large: esm2_t33_650M_UR50D

 # ElnaggarLab/ankh-base
 ### https://github.com/agemagician/Ankh/blob/main/examples/binary_classification_solubility_task.ipynb - different model?
 #  https://github.com/agemagician/Ankh#models   - 450M model size # model_checkpoint =   "ElnaggarLab/ankh-base"

model_max_len = min(1024,MAX_LEN) # 800

# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint,
# #                                           padding= False#True # orig
#                                           padding= True# alt
#                                           ,truncation=True,max_length=model_max_len)

num_labels = 2 # max(train_labels + test_labels) + 1  # Add 1 since 0 can be a label
print("Num labels:", num_labels)
##ORIG:
# model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels,problem_type="single_label_classification") # worked, orig

## try this now, alt:
# model = EsmForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
# model = TFEsmForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels,problem_type="single_label_classification")

## https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/protein_language_modeling.ipynb#scrollTo=fc164b49 # uses trainer
# ##
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, #num_labels=num_labels,
# #                                                            problem_type="single_label_classification", # was enabled
# #                                                            load_in_4bit=True, # disable to train ok
# #                                                            load_in_4bit= False,
# #                                                             quantization_config=nf4_config,
#                                                           #  load_in_8bit=True,  torch_dtype=torch.float32, # try this - new
# #                                                            device_map= "cuda:0",#"auto",
#                                                            device_map="auto",
#                                                           num_labels=len(ID2LABEL), id2label=ID2LABEL, label2id=LABEL2ID,
# #                                                             trust_remote_code=True,
#                                                           )
# model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True) # prepares the whole model for kbit training

# last_layer_num = model.num_layers ## 33 for esm2_t33_650M_UR50D
# print(last_layer_num )

# model.print_trainable_parameters()

Num labels: 2


In [23]:
def get_raw_model_lora(model_checkpoint):
    model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,
                                                           problem_type="single_label_classification", # was enabled
#                                                            load_in_4bit=True,
#                                                             quantization_config=nf4_config,
                                                          #  load_in_8bit=True,  torch_dtype=torch.float32, # try this - new
                                                           device_map="auto",
                                                          num_labels=len(ID2LABEL), id2label=ID2LABEL, label2id=LABEL2ID,
                                                          )
    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True) # prepares the whole model for kbit training
    peft_config = LoraConfig(base_model_name_or_path=model_checkpoint,
    task_type= "SEQ_CLS",inference_mode=False, r= 2 if FAST_RUN else 8, #16,
    lora_alpha=8,use_rslora = True, bias= "all"  ,target_modules=  "all-linear"
    # # ### https://www.kaggle.com/code/andregrbnr/protein-sequence-classification
#      ,modules_to_save= "all-linear",
#                          [#"decode_head",
# #                       "classifier",
#          "EsmClassificationHead",
#          'classifier.dense', 'classifier.out_proj',
#          "pooler",
# # #                      'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight'
#                      ]
)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model

In [24]:
# model = get_raw_model_lora(model_checkpoint)

"After we wrap our base model model with PeftModel along with the config, we get a new model where only the LoRA parameters are trainable (so-called “update matrices”) while the pre-trained parameters are kept frozen. These include the parameters of the randomly initialized classifier parameters too. This is NOT we want when fine-tuning the base model on our custom dataset. To ensure that the classifier parameters are also trained, we specify modules_to_save. This also ensures that these modules are serialized alongside the LoRA trainable parameters when using utilities like save_pretrained() and push_to_hub().

In addition to specifying the target_modules within LoraConfig, we also need to specify the modules_to_save. When we wrap our base model with PeftModel and pass the configuration, we obtain a new model in which only the LoRA parameters are trainable, while the pre-trained parameters and the randomly initialized classifier parameters are kept frozen. However, we do want to train the classifier parameters. By specifying the modules_to_save argument, we ensure that the classifier parameters are also trainable, and they will be serialized alongside the LoRA trainable parameters when we use utility functions like save_pretrained() and push_to_hub()."

In [25]:
## Warning - longer than allowed length - 1024
# """

# same tokenizer for all?
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t12_35M_UR50D",
#                                           padding= False#True # orig
                                          padding= True# alt
                                          ,truncation=True,max_length=model_max_len)

train_tokenized = tokenizer(train_sequences,  truncation=True,max_length=model_max_len,padding=True) # padding=True,
test_tokenized = tokenizer(test_sequences,  truncation=True,max_length=model_max_len,padding=True) # padding=True,

train_dataset = Dataset.from_dict(train_tokenized)
test_dataset = Dataset.from_dict(test_tokenized)

train_dataset = train_dataset.add_column("labels", train_labels)
test_dataset = test_dataset.add_column("labels", test_labels)
train_dataset
# """

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 20340
})

In [26]:
# from accelerate import Accelerator
# # # Use the accelerator
# # ### try disabling? (with qlora)
# # accelerator = Accelerator()# trying this
# # # # accelerator = Accelerator(mixed_precision="fp16") # fp16 # orig used
# model = accelerator.prepare(model)

train_dataset = accelerator.prepare(train_dataset)
test_dataset = accelerator.prepare(test_dataset)

With 35M model :
```
Default (1024) max length
batch_size=8

opt = Adafactor(1e-4)##AdamWeightDecay(1e-4) #default: AdamWeightDecay(2e-5)
model.compile(optimizer=opt, metrics=["accuracy"],
              loss="BinaryCrossentropy")
3813/3813 [==============================] - 2625s 661ms/step - loss: 0.2452 - accuracy: 0.9104 - val_loss: 0.1622 - val_accuracy: 0.9363
Epoch 2/3
3813/3813 [==============================] - 2518s 661ms/step - loss: 0.1218 - accuracy: 0.9597 - val_loss: 0.1533 - val_accuracy: 0.9463
Epoch 3/3
3813/3813 [==============================] - 2523s 662ms/step - loss: 0.0730 - accuracy: 0.9799 - val_loss: 0.1978 - val_accuracy: 0.9436

```

In [27]:
from sklearn.metrics import precision_recall_fscore_support,accuracy_score,roc_auc_score
import evaluate

from datasets import load_metric
import torch._dynamo
torch._dynamo.config.suppress_errors = True

metric_acc = evaluate.load("accuracy")
metric = evaluate.load("accuracy")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        "roc_auc":roc_auc_score(labels,pred.predictions[:,1])
    }

In [28]:
# if TRAIN_MODEL:
#     result = trainer.train()

In [29]:
models_list

['facebook/esm2_t6_8M_UR50D',
 'facebook/esm2_t12_35M_UR50D',
 'facebook/esm2_t30_150M_UR50D',
 'facebook/esm2_t33_650M_UR50D']

In [30]:
result_eval_dict = {}

## could log models vs run properly, skip for now
## https://docs.wandb.ai/guides/track/log/log-models

for model_checkpoint in models_list:
    # ### AttributeError: 'TFEsmForSequenceClassification' object has no attribute 'to'
    model_name = model_checkpoint.split("/")[-1]
    print(model_name)

    training_args = TrainingArguments(
        run_name=f"{model_name}-finetuned-ft-humVir",
    #     f"/content/drive/MyDrive/proteins/New Protein-Virus anom project/t{model_name}-finetuned-humVir",
        per_device_train_batch_size=bch_size,
        overwrite_output_dir=True,
        # per_device_eval_batch_size=int(2*bch_size),
        # gradient_accumulation_steps= 2, #4,
        gradient_checkpointing=True, # maybe causes slowdown?
        # fp16=True,
        bf16=True, # needs ampere, not supported
        tf32=True,
            torch_compile = True,
        optim = "adamw_8bit",#"paged_adamw_8bit", # adamw_bnb_8bit
    #     optim="paged_adamw_32bit",
        label_names = ["labels"],
        learning_rate = 5e-4,#5e-4, #5e-3,
        # lr_scheduler_type="cosine",
        max_grad_norm = 0.95,
    #     weight_decay=0.001,
        # eval_accumulation_steps = 2#8
        evaluation_strategy="epoch",
        # warmup_ratio=0.02,
        save_strategy= "epoch",#"no",
        output_dir=".",
         no_cuda=False,
         greater_is_better=True,
         # save_total_limit=1,
      remove_unused_columns=False,
        auto_find_batch_size = True, # new , reduces if oom
        num_train_epochs=num_epochs,
        # load_best_model_at_end=True,
        metric_for_best_model= "eval_roc_auc",#"accuracy",
        group_by_length=True,

        report_to="wandb",  # enable logging to W&B
        # run_name="bert-base-high-lr",  # name of the W&B run (optional)
        # logging_steps=1,  # how often to log to W&B
    )

    model = get_raw_model_lora(model_checkpoint)
    trainer = Trainer(model=model, args=training_args,
                  train_dataset=train_dataset,eval_dataset=test_dataset,
                  tokenizer=tokenizer,
                  compute_metrics=compute_metrics,)
    train_result = trainer.train()
    result_eval_dict[model_name] =  train_result.metrics
    result_eval_dict[model_name].update(trainer.evaluate())
    # result_eval_dict[model_name] = trainer.evaluate()
print(result_eval_dict[model_name])

esm2_t6_8M_UR50D


Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 409,947 || all params: 8,236,965 || trainable%: 4.976918076014648


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


/home/ddofer/anaconda3/envs/hf/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


...

trainable params: 8,226,747 || all params: 660,148,065 || trainable%: 1.2461972451589327


/home/ddofer/anaconda3/envs/hf/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Roc Auc
1,0.104000,0.075292,0.971740,0.958387,0.955699,0.961142,0.995090
2,0.044200,0.090054,0.974042,0.962364,0.953097,0.972507,0.995853
3,0.016300,0.085569,0.978648,0.968482,0.966849,0.970138,0.996657


Checkpoint destination directory ./checkpoint-1272 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/home/ddofer/anaconda3/envs/hf/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
Checkpoint destination directory ./checkpoint-2544 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/home/ddofer/anaconda3/envs/hf/lib/python3.10/site-packages/peft/utils/other.py:588: UserWarning: Unable to fetch remote file due to the following error (MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded wit

{'train_runtime': 16903.0131, 'train_samples_per_second': 3.61, 'train_steps_per_second': 0.226, 'total_flos': 2.469858576132096e+17, 'train_loss': 0.06412847974265397, 'epoch': 3.0, 'eval_loss': 0.0855686143040657, 'eval_accuracy': 0.9786476868327402, 'eval_f1': 0.968481618902723, 'eval_precision': 0.9668493030595249, 'eval_recall': 0.9701380071405388, 'eval_roc_auc': 0.996656799740344, 'eval_runtime': 335.5286, 'eval_samples_per_second': 14.237, 'eval_steps_per_second': 1.782}


In [31]:
result_eval_dict[model_name] =  train_result.metrics
result_eval_dict[model_name].update(trainer.evaluate())
# result_eval_dict[model_name] = trainer.evaluate()

In [32]:
result_eval = pd.DataFrame(result_eval_dict).T
display(result_eval)
if SAVE_OUTPUTS:
    result_eval.to_csv(f"DL_eval_{today}.csv")

,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss,epoch,eval_loss,eval_accuracy,eval_f1,eval_precision,eval_recall,eval_roc_auc,eval_runtime,eval_samples_per_second,eval_steps_per_second
esm2_t6_8M_UR50D,1588.6776,38.409,2.402,2.961046e+15,0.157278,3.0,0.203130,0.947247,0.921523,0.923343,0.919735,0.980877,31.2410,152.908,19.142
esm2_t12_35M_UR50D,3539.3568,17.240,1.078,1.296121e+16,0.109685,3.0,0.184896,0.958342,0.938138,0.939233,0.937054,0.986867,67.5681,70.699,8.850
esm2_t30_150M_UR50D,9839.0309,6.202,0.388,5.673499e+16,0.077224,3.0,0.120590,0.969856,0.955409,0.954769,0.956053,0.992572,186.6819,25.589,3.203
esm2_t33_650M_UR50D,16903.0131,3.610,0.226,2.469859e+17,0.064128,3.0,0.085569,0.978648,0.968482,0.966849,0.970138,0.996657,335.2372,14.250,1.784


In [33]:
wandb.finish()

eval/accuracy,▁▄▅▅▅▅▆▆▆▇▇▇▇████
eval/f1,▁▄▅▅▄▅▆▆▆▇▇▇▇████
eval/loss,█▅▆▆▅▄▆▆▂▂▃▃▁▂▁▁▁
eval/precision,▁▄▅▅▅▆▆▆▆▇▇▇▇▇███
eval/recall,▁▁▂▂▂▃▄▄▆▅▆▆▇████
eval/roc_auc,▁▃▃▃▄▅▅▅▆▇▇▇█████
eval/runtime,▁▁▁▁▂▂▂▂▅▅▅▅█████
eval/samples_per_second,████▄▄▄▄▂▂▂▂▁▁▁▁▁
eval/steps_per_second,████▄▄▄▄▂▂▂▂▁▁▁▁▁
train/epoch,▁▂▃▃▄▅▆▇██▁▂▃▄▅▅▆▇██▁▂▃▄▅▅▇███▁▃▃▄▅▆▇███
train/global_step,▁▂▃▃▄▅▆▇██▁▂▃▄▅▅▆▇██▁▂▃▄▅▅▇███▁▃▃▄▅▆▇███


In [36]:
result_eval.to_csv(f"DL_eval_{today}.csv")

### run CV

150M , full data, withgradient checkpoint enabled: 2.2 min per epoch (4.4 total), for 500 (long) samples.
```
 [42/42 04:41, Epoch 2/2]
Epoch	Training Loss	Validation Loss	Accuracy	F1	Precision	Recall	Roc Auc
1	No log	0.344198	0.868263	0.868263	0.868263	0.868263	0.938507
2	No log	0.255124	0.880240	0.880240	0.880240	0.880240	0.958227
```
similar time when not using gradient checkpoint and double (32) batch szie - 4 min for. epochs?